### Finetune GPT-4o Vision

In [ ]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

In [7]:
image_path = "/Users/abdulrahmanalrabah/Desktop/UIUC/Fall 2024/CS 543/project/projective-geometry/dataset/Kandinsky_Indoor/test/real/99000.jpg"

In [8]:
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

base64_image = encode_image(image_path)
image_url  = f"data:image/jpeg;base64,{base64_image}"

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image, answer in one senetence?"},
        {
          "type": "image_url",
          "image_url": {
            "url": image_url,
          },
        },
      ],
    }
  ],
  max_tokens=100,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image shows a modest bedroom featuring a bed with floral bedding, a chair, and a window with lace curtains.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [ ]:
import glob
import json
import os
import base64

def encode_image(image_path):
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        return None
    except Exception as e:
        print(f"Error encoding image {image_path}: {e}")
        return None

real_images_path = "/Users/abdulrahmanalrabah/Desktop/UIUC/Fall 2024/CS 543/project/projective-geometry/vision_finetuning/Kandinsky_indoor/Kandinsky_indoor_real/*"
fake_images_path = "/Users/abdulrahmanalrabah/Desktop/UIUC/Fall 2024/CS 543/project/projective-geometry/vision_finetuning/Kandinsky_indoor/Kandinsky_indoor_fake/*"

# Output JSONL file
output_file = "/Users/abdulrahmanalrabah/Desktop/UIUC/Fall 2024/CS 543/project/projective-geometry/vision_finetuning/Kandinsky_indoor/real_fake_train.jsonl"

processed_count = {"real": 0, "fake": 0}

with open(output_file, "w") as fp:
    for image_path in glob.iglob(real_images_path):
        base64_image = encode_image(image_path)
        if base64_image is None:
            continue
        
        image_url = f"data:image/jpeg;base64,{base64_image}"

        ground_truth = {"label": "real"}

        example = {
            "messages": [
                {"role": "system", 
                 "content": "You are an assistant that classifies images as real or fake."},
                {"role": "user", 
                 "content": "Classify the image as either real or fake."},
                {"role": "user", 
                 "content": [
                     {
                         "type": "image_url",
                         "image_url": {
                             "url": image_url
                         }
                     }
                 ]},
                {"role": "assistant", "content": f"{ground_truth}"}
            ]
        }

        example_str = json.dumps(example)
        fp.write(example_str + "\n")
        processed_count["real"] += 1

    # Process fake images
    for image_path in glob.iglob(fake_images_path):
        base64_image = encode_image(image_path)
        if base64_image is None:
            continue
        
        image_url = f"data:image/jpeg;base64,{base64_image}"

        ground_truth = {"label": "fake"}

        example = {
            "messages": [
                {"role": "system", 
                 "content": "You are an assistant that classifies images as real or fake."},
                {"role": "user", 
                 "content": "Classify the image as either real or fake."},
                {"role": "user", 
                 "content": [
                     {
                         "type": "image_url",
                         "image_url": {
                             "url": image_url
                         }
                     }
                 ]},
                {"role": "assistant", "content": f"{ground_truth}"}
            ]
        }

        example_str = json.dumps(example)
        fp.write(example_str + "\n")
        processed_count["fake"] += 1

print(f"Processed {processed_count['real']} real images and {processed_count['fake']} fake images.")
print(f"Output file: {output_file}")


Processed 20 real images and 20 fake images.
Output file: /Users/abdulrahmanalrabah/Desktop/UIUC/Fall 2024/CS 543/project/projective-geometry/vision_finetuning/Kandinsky_indoor/real_fake_train.jsonl
